In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup # commonly used for parsing HTML content

import re
import spacy
from spacy.tokens import DocBin

In [3]:
# Get resume links from link of pages
def get_resume(url_web='https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer'):
    href_list = []
    print(f'The function takes 30 pages, which is the maximum at the present time of {url_web}')

    for j in range(1, 30):
        url_page = url_web + f'/page_{j}'

        response = requests.get(url_page)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all links that we need on the page
        href_locals = soup.find_all('a', attrs={'href': True}, href=lambda value: value.startswith('/job/view-resume-'))
        try:
            for href_local in href_locals:
                href_list.append('https://www.jobspider.com' + href_local['href'])
        except:
            pass

    return href_list

# Convert resume into text
def resume_to_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')

    # Get job function sought
    resume_title = soup.find_all(["h1"])[0].text.strip()

    # Get job decription
    decriptions = soup.find_all(["td"])
    return url, resume_title, [decription.text.strip() for decription in decriptions]

In [4]:
def preprocess(decription):
    text = "\n".join(decription[19:-3])                                   # omit data after viz
    text = re.sub(r'\n\s*\n', '\n', text)                                 # Replace multiple newlines with one
    text = re.sub(r'\s{2,}', ' ', text)                                   # Replace multiple spaces with one
    text = text.replace('\t', ' ')                                        # Replace tabs with one space
    text = "".join([s for s in text.splitlines(True) if s.strip("\r\n")]) # filters out any empty lines
    text = re.sub('[^A-Za-z0-9\n]+', ' ', text)                           # replace any characters that are not alphanumeric

    return text

In [5]:
href_list = get_resume() # default is https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer

# List to store information
urls = []
resume_types = []
decriptions = []

data = []
nlp_temp = spacy.load("en_core_web_lg")

for href in tqdm(href_list):
    url, resume_title, description = resume_to_text(href)

    _description = preprocess(description) # preprocessing base on obseravtion
    doc = nlp_temp(_description) # fot getting ner in _description
    row = [url, resume_title, _description, {'entities': [(ent.start_char, ent.end_char, ent.text, ent.label_, str(spacy.explain(ent.label_))) for ent in doc.ents]}]

    data.append(row)

The function takes 30 pages, which is the maximum at the present time of https://www.jobspider.com/job/resume-search-results.asp/words_Software%2BEngineer


100%|██████████| 1450/1450 [14:02<00:00,  1.72it/s]


In [13]:
import json
with open('/kaggle/working/scrapping_data.json', 'w') as f:
    json.dump(data, f)

In [14]:
with open("/kaggle/working/scrapping_data.json", "rb") as f:
    json_data = json.load(f)
    
json_data[0]

['https://www.jobspider.com/job/view-resume-84755.html',
 'Marketing Manager/Sr. Marketing Manager, Partnet Marketing Manager, Brand Manage Resume',
 'Marketing Manager Sr Marketing Manager Partnet Marketing Manager Brand Manage Resume\nDesired Industry Marketing\nSpiderID 84755\nDesired Job Location Marietta Georgia\nDate Posted 10 7 2024\nType of Position Full Time Permanent\nAvailability Date Immediately\nDesired Wage 100000\nU S Work Authorization Yes\nJob Level Management Manager Director \nWilling to Travel Yes Less Than 25 \nHighest Degree Attained Bachelors\nWilling to Relocate No\nObjective Success driven and highly creative professional with extensive experience in marketing management complemented with expertise in developing innovative marketing strategies and campaigns that drove brand awareness and market penetration Noted for proven skills in developing and executing strategic marketing initiatives focused on generating sales and meeting revenue targets Expert at capturi

In [7]:
import pickle
pickle.dump(data, open('scrapping_data.pkl','wb'))

In [8]:
with open("scrapping_data.pkl", "rb") as f:
    pkl_data = pickle.load(f)
    
pkl_data[0]

['https://www.jobspider.com/job/view-resume-84755.html',
 'Marketing Manager/Sr. Marketing Manager, Partnet Marketing Manager, Brand Manage Resume',
 'Marketing Manager Sr Marketing Manager Partnet Marketing Manager Brand Manage Resume\nDesired Industry Marketing\nSpiderID 84755\nDesired Job Location Marietta Georgia\nDate Posted 10 7 2024\nType of Position Full Time Permanent\nAvailability Date Immediately\nDesired Wage 100000\nU S Work Authorization Yes\nJob Level Management Manager Director \nWilling to Travel Yes Less Than 25 \nHighest Degree Attained Bachelors\nWilling to Relocate No\nObjective Success driven and highly creative professional with extensive experience in marketing management complemented with expertise in developing innovative marketing strategies and campaigns that drove brand awareness and market penetration Noted for proven skills in developing and executing strategic marketing initiatives focused on generating sales and meeting revenue targets Expert at capturi

In [18]:
def get_spacy_doc(file, data):

  nlp = spacy.blank("en")
  db = DocBin()

  for url, resume_tittle, text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    for start, end, text, label, explain in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label = label, alignment_mode = 'strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + "    " + str(text) + '\n'
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [19]:
from sklearn.model_selection import train_test_split
train, dev = train_test_split(json_data, test_size = 0.2, random_state = 42)

In [20]:
file = open('train_file.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train.spacy')

db = get_spacy_doc(file, dev)
db.to_disk('dev.spacy')

file.close()

100%|██████████| 290/290 [00:22<00:00, 12.91it/s]


References from some bellow cell code blocks is taken from https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647

In [23]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 84.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [24]:
# Initialization setting
!python -m spacy init fill-config /kaggle/input/up-source/base_config.cfg config.cfg

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [25]:
# This took approximately 8 hour for training on google colab pro gpu mode
!python -m spacy train -g 0 config.cfg  --output /kaggle/working/ --paths.train train.spacy --paths.dev dev.spacy 

ℹ Saving to output directory: /kaggle/working
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
[2024-10-07 16:31:22,981] [INFO] Set up nlp object from config
[2024-10-07 16:31:23,022] [INFO] Pipeline: ['transformer', 'ner']
[2024-10-07 16:31:23,028] [INFO] Created vocabulary
[2024-10-07 16:31:23,028] [INFO] Finished initializing nlp object
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download`

In [28]:
!pip install spacy-transformers

In [30]:
import spacy_transformers

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [35]:
model = spacy.load('/kaggle/working/model-best')

In [32]:
# Test example
main_str = ['''Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response
Manufacturing Production Manager Resume
Desired Industry Manufacturing
SpiderID 78692
Desired Job Location Windsor Colorado
Date Posted 1 23 2017
Type of Position Full Time Permanent
Availability Date Immediately
Desired Wage 95000
U S Work Authorization Yes
Job Level Management Manager Director
Willing to Travel
Highest Degree Attained Other
Willing to Relocate Yes
Objective Dynamic and growth driven professional offering hands on management experience and comprehensive background in manufacturing and engineering operations within highly competitive setting Adept at reengineering unproductive work processes as well as in planning and implementing various sustainable and cost effective work programs to drive continuous improvement of operations Armed with exceptional organizational and critical problem solving aptitudes to formulate effective solutions on complex production and quality issues Equipped with tactical leadership capabilities in supervising and guiding teams toward the successful and timely completion of projects Proficient with Microsoft Office applications Project Management and Microsoft Visio
Experience Relevant ExperienceVestas Blades America Windsor COProduction Engineer Jan 2014 PresentContribute efforts in achieving production plan budget tooling equipment and bill of material along with quality control and safety Conceptualized new methods to optimize production levels while maintaining production costs yields quality and safety Identified and resolved process problems with effective solutions which decreased downtime and minimized costs Initiated plant trials to measure performance capabilities while ensuring updated documentation of process procedures Leveraged industry expertise in streamlining the manufacturing of turbine blades Production Supervisor Mar 2009 Jan 2014Rendered oversight to more than 70 employees to guarantee accordance of operation with production plan and goals Observed strict compliance with safety and quality guidelines and handled inventory control created schedules delegated work and facilitated training of staff Generated production reports for production and operations managers regarding production areas performance Served as a Shells Lighthouse project member while drafting and modifying all standard operating procedures Functioned as department lead for production quality training and process improvements Closely monitored operations productivity to determine areas for improvement in overall production process Pioneered the development of all shells production process job cards that decreased process times and improved efficiency and quality through changed production flow layout Anheuser Busch Fort Collis COBrew House Control Panel Operator Mar 2007 Mar 2009Efficiently administered beer brewing process from raw material selection and recipe formulation through the use of Siemens software Strictly enforced standard operating procedures and safe working practices Determined and evaluated all critical control points to achieve consistent product quality within allotted schedules Other ExperienceUnited States Airways Denver CoCustomer Service Representative Kroger Co King Soopers Smiths Food Drug Fort Collins CoGrocery Manager Head Clerk Night Crew Manager Front End ManagerInventory Control Manager Warehouse Manager
Education EducationAssociate of Science with emphasis in chemistry and biologyFront Range Community College Fort Collins COPharmacy Pre Requisites for Doctor of Pharmacy Pharm D University of Wyoming Laramie WYPharmacy Pre Requisites for Doctor of Pharmacy Pharm D Western Wyoming Community College Rock Springs WY Deans Honor RollProfessional DevelopmentCertificationsSupervisor Certification Six Sigma Yellow Belt Project ManagementQuality Management Crucial Conversations Fort Lift License Crane License
Affiliations
Skills Manufacturing Production Manager Project Planning Cost Reduction and Budget Optimization Resource Allocation Six Sigma Quality ControlLean Manufacturing Plant Management Manufacturing Inspection Administration Cross functional Team Building
Additional Information TrainingProduction Instructor Coordinator Planner Coordinator Wrote Training Document
Reference Available upon request
Candidate Contact Information
JobSpider com has chosen not to make contact information available on this page Click Contact Candidate to send this candidate a response''' ]


In [36]:
for doc in model.pipe(main_str, disable=["tagger", "parser"]):
  for ent in doc.ents:
    print((ent.text,ent.label_))
    # print([(ent.text, ent.label_) for ent in doc.ents])


('Resume\nDesired Industry Manufacturing', 'ORG')
('78692', 'CARDINAL')
('Colorado', 'GPE')
('23 2017', 'DATE')
('95000', 'CARDINAL')
('U S Work Authorization', 'PRODUCT')
('Microsoft Office', 'ORG')
('Project Management', 'ORG')
('Microsoft Visio', 'ORG')
('Jan 2014', 'DATE')
('PresentContribute', 'ORG')
('Mar 2009 Jan', 'DATE')
('2014Rendered', 'CARDINAL')
('more than 70', 'CARDINAL')
('Shells Lighthouse', 'ORG')
('Anheuser Busch Fort Collis COBrew House Control Panel Operator', 'ORG')
('Mar 2007 Mar', 'DATE')
('Siemens', 'ORG')
('ExperienceUnited States Airways', 'ORG')
('Kroger Co', 'ORG')
('Education EducationAssociate of Science', 'ORG')
('biologyFront Range Community College', 'ORG')
('Western Wyoming Community College', 'ORG')
('Rock Springs', 'GPE')
('Affiliations\nSkills Manufacturing Production', 'ORG')
('Six', 'CARDINAL')
('Candidate Contact Information\n', 'ORG')
('Click Contact Candidate', 'WORK_OF_ART')
('Manufacturing Production', 'ORG')
('78692', 'CARDINAL')
('Colorado